In [1]:
import numpy as np
import sys
sys.path.append("..")
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
import sys
from data_loaders import *
from missing_process.block_rules import *
import json
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

from tqdm import tqdm

from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import Ridge
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import f1_score
from sklearn.cluster import DBSCAN
from sklearn.metrics import (
    homogeneity_score,
    completeness_score,
    v_measure_score,
)

from sklearn.metrics import (
    mutual_info_score,
    normalized_mutual_info_score,
    adjusted_mutual_info_score,
)

In [2]:
# cluster_datalist = [
#     "banknote",
#         "concrete_compression",
#             "climate_model_crashes",
#             "connectionist_bench_sonar","qsar_biodegradation",
#             "yeast"
#             ]

real_datalist = [
    "banknote",
        "concrete_compression",
            "wine_quality_white","wine_quality_red",
            "california","climate_model_crashes",
            "connectionist_bench_sonar","qsar_biodegradation",
            "yeast","yacht_hydrodynamics"
            ]


# except_list = ["banknote"
# "climate_model_crashes"
# "connectionist_bench_sonar"]

real_datalist = [
        "concrete_compression",
            "wine_quality_white","wine_quality_red",
            "california",
"qsar_biodegradation",
            "yeast","yacht_hydrodynamics"
            ]

missingtypelist = [
                    "quantile",
                   "diffuse",
                   "logistic"
                   "mcar","mar"
                   ]

seed = 1
nfold = 5

In [3]:
def load_impute_data(missingtype,model_name,rule_name,dataname,fold,seed = 1):

    if model_name == "tabcsdi" and missingtype in ["mcar","mar"]:
        train_impute = np.load(f'impute_hpc/{missingtype}/{dataname}/{model_name}/{rule_name}_seed-{seed}_{fold}_train.npy')
        test_impute = np.load(f'impute_hpc/{missingtype}/{dataname}/{model_name}/{rule_name}_seed-{seed}_{fold}_test.npy')
        return train_impute,test_impute

    train_impute = np.load(f'impute/{missingtype}/{dataname}/{model_name}/{rule_name}_seed-{seed}_{fold}_train.npy')
    test_impute = np.load(f'impute/{missingtype}/{dataname}/{model_name}/{rule_name}_seed-{seed}_{fold}_test.npy')
    return train_impute,test_impute

def load_train_test(index_file,norm_values,observed_masks):  

    train_index = index_file["train_index"]
    test_index = index_file["test_index"]

    train_values = norm_values[train_index,:]

    train_masks = observed_masks[train_index,:]

    test_values = norm_values[test_index,:]

    test_masks = observed_masks[test_index,:]


    return train_values,train_masks,test_values,test_masks

In [4]:
def fillin_imputed_data(imputed,mask,original):
    filled_data = np.where(mask == 1, original, imputed)
    return filled_data

In [5]:
# Single Model Name

datalist = real_datalist
model_name = "tabcsdi"
missingtypelist = ["logistic"]
for missingtype in missingtypelist:
    if missingtype == "logistic":
        missing_rule = load_json_file("missing_rate.json")
    elif missingtype == "diffuse":
        missing_rule = load_json_file("diffuse_ratio.json")
    elif missingtype == "quantile":
        missing_rule = load_json_file("quantile_full.json")
    elif missingtype == "mcar" or missingtype == "mar":
        missing_rule = load_json_file("mcar.json")
    d_v = {}
    d_AMI = {}

    # load data and its mask
    for dataname in tqdm(datalist):
        
        directory_path = f"datasets/{dataname}"
        data = dataset_loader(dataname)    
        norm_values = np.load(f'{directory_path}/{dataname}_norm.npy')

        # clustering = DBSCAN().fit(norm_values)
        # #print(clustering.labels_)
        # ncluster = len(set(clustering.labels_))
        # if ncluster == 1:
        #     print(dataname)
        d_v[dataname] = []
        d_AMI[dataname] = []

        for rule_name in tqdm(missing_rule):
            
            observed_masks = np.load(f'{directory_path}/{missingtype}/{rule_name}.npy')
            f = open(f'{directory_path}/split_index_cv_seed-{seed}_nfold-{nfold}.json')
            index_file = json.load(f)

            v_list = []
            AMI_list = []
            for fold in index_file:
                index = index_file[fold]
                train_values,train_masks,test_values,test_masks = load_train_test(index,norm_values,observed_masks)
                impute_train,impute_test  = load_impute_data(missingtype,model_name,rule_name,dataname,fold)

                impute_test = fillin_imputed_data(impute_test,test_masks,test_values)
                # print(impute_test)
                # print(test_values)
                # print(test_masks)
                clu_test = DBSCAN().fit(test_values).labels_
                clu_test_imp = DBSCAN().fit(impute_test).labels_

                
                V = v_measure_score(clu_test, clu_test_imp)
                AMI = adjusted_mutual_info_score(clu_test, clu_test_imp)

                v_list.append(V)
                AMI_list.append(AMI)
            

            d_v[dataname].append(np.mean(v_list))
            d_AMI[dataname] .append(np.mean(AMI_list))

    df_v = pd.DataFrame(d_v,index=[rule_name for rule_name in missing_rule])
    df_AMI = pd.DataFrame(d_AMI,index=[rule_name for rule_name in missing_rule])

    path = f"clustering/{missingtype}/"
    if not os.path.exists(path):
        os.makedirs(path)

    df_v.to_csv(f"{path}/{model_name}_V.csv")

    df_AMI.to_csv(f'{path}/{model_name}_AMI.csv')



  0%|          | 0/7 [00:00<?, ?it/s]

 43%|████▎     | 3/7 [00:13<00:18,  4.67s/it]


KeyboardInterrupt: 

In [6]:
def fillin_imputed_data(imputed, mask, original):
    """
    Fill in missing values in the imputed dataset using the original dataset and the mask.
    Replace NaNs in the imputed dataset with 1 when the mask indicates missing values.

    Parameters:
        imputed (numpy.ndarray): Imputed dataset with missing values.
        mask (numpy.ndarray): Mask indicating missing values (NaNs).
        original (numpy.ndarray): Original dataset with complete values.

    Returns:
        numpy.ndarray: New dataset with missing values filled in.
    """
    filled_data = np.where(np.isnan(mask), 1, imputed)
    return filled_data

In [8]:

datalist = real_datalist


def run_cluster(rule_name,model_name_list,missingtype,datalist):

    d_v = {}
    d_AMI= {}

    for dataname in tqdm(datalist):
        
        directory_path = f"datasets/{dataname}"
        data = dataset_loader(dataname)    
        norm_values = np.load(f'{directory_path}/{dataname}_norm.npy')
        
        observed_masks = np.load(f'{directory_path}/{missingtype}/{rule_name}.npy')
        f = open(f'{directory_path}/split_index_cv_seed-{seed}_nfold-{nfold}.json')
        index_file = json.load(f)

        d_v[dataname] = {}
        d_AMI[dataname] = {}

        for model_name in model_name_list:
            d_v[dataname][model_name]=0
            d_AMI[dataname][model_name]=0
            # load data and its mask

            v_list = []
            AMI_list = []
            for fold in index_file:
                index = index_file[fold]
                train_values,train_masks,test_values,test_masks = load_train_test(index,norm_values,observed_masks)
                impute_train,impute_test  = load_impute_data(missingtype,model_name,rule_name,dataname,fold)

                impute_test = fillin_imputed_data(impute_test,test_masks,test_values)
                impute_test = np.nan_to_num(impute_test, nan=0)

                clu_test = DBSCAN().fit(test_values).labels_
                clu_test_imp = DBSCAN().fit(impute_test).labels_

                
                V = v_measure_score(clu_test, clu_test_imp)
                AMI = adjusted_mutual_info_score(clu_test, clu_test_imp)

                v_list.append(V)
                AMI_list.append(AMI)
            

            d_v[dataname][model_name]=np.mean(v_list)
            d_AMI[dataname][model_name]=np.mean(AMI_list)


    df_v = pd.DataFrame(d_v).T
    df_AMI = pd.DataFrame(d_AMI).T
    new_col_names = ["RD", "ZR", "Mean", "KNN", "MF", "Mice", "MisF", "XGB", "OT", "HI", "GAIN", "Mi", "NMi", "CSDI"]
    new_row_names = ["Bank","Cali","Climate","Concre","Qsar",  "Red",  "Sonar", "White", "Yacht","Yeast"]

    path = f"clustering/{missingtype}/"
    if not os.path.exists(path):
        os.makedirs(path)

    df_v = df_v.rename(index=dict(zip(df_v.index, new_row_names)), columns=dict(zip(df_v.columns, new_col_names)))
    df_AMI = df_AMI.rename(index=dict(zip(df_AMI.index, new_row_names)), columns=dict(zip(df_AMI.columns, new_col_names)))

    df_v.to_csv(f"{path}/{rule_name}_V.csv")

    df_AMI.to_csv(f'{path}/{rule_name}_AMI.csv')


In [9]:
model_name_list = ["random", "zero", "mean", "knn", "mf", "mice", "missforest", "XGB", "ot", "hyper", "gain", "miwae", "notmiwae", "tabcsdi"]

# real_datalist = [
#         "concrete_compression",
#             "wine_quality_white","wine_quality_red",
#             "california",
# "qsar_biodegradation",
#             "yeast","yacht_hydrodynamics"
#             ]


real_datalist = ["banknote",
            "california","climate_model_crashes","concrete_compression",
           "qsar_biodegradation","wine_quality_red", "connectionist_bench_sonar","wine_quality_white",
            "yacht_hydrodynamics","yeast"
            ]

for missingtype in ["diffuse","logistic"]:
    for rule_name in ["0.3","0.5","0.7"]:
        run_cluster(rule_name,model_name_list,missingtype,real_datalist)


100%|██████████| 10/10 [00:49<00:00,  4.97s/it]


In [10]:
for missingtype in ["mar","mcar"]:
    for rule_name in ["0.3","0.5","0.7"]:
        run_cluster(rule_name,model_name_list,missingtype,real_datalist)

100%|██████████| 10/10 [00:42<00:00,  4.21s/it]


In [11]:
missingtype = "quantile"
for rule_name in ["Q1_Q4_0.5","Q2_Q3_0.5","Q2_Q4_0.5"]:
    run_cluster(rule_name,model_name_list,missingtype,real_datalist)

100%|██████████| 10/10 [00:52<00:00,  5.22s/it]
